In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 04 15:26:55 2019
@author: jianwuwang, saviokay
"""

'\nCreated on Mon Mar 04 15:26:55 2019\n@author: jianwuwang, saviokay\n'

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import time
import math
import dask.array as da
import h5py
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

In [3]:
# Setting File Location As Environment Variables
MOD03_path =sys.argv[1] #'HDFFiles/'
MOD03_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
MOD06_path =sys.argv[2] #'HDFFiles/'
MOD06_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'

In [4]:
satellite = 'Aqua'
yr = [2008]
mn = [1]
dy = [1]
# Latitude & Longtitude Boundaries Of Level-3 Grid
lat_bnd = np.arange(-90,91,1)
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

In [5]:
# Read Function For MODO3 & MODO6 Files.
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    #print('Reading The Cloud Mask From MOD06_L2 Product:')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
   # print('The Level-2 Cloud Mask Array Shape',CM.shape)
    print(' ')

    myd03 = Dataset(MOD03_file, "r")
    #print('Reading The Latitude-Longitude From MOD03 Product:')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #print('The Level-2 Latitude-Longitude Array Shape',latitude.shape)
    print(' ')

    return latitude,longitude,CM

# Misc Function For Processing Cloud Fraction.
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')

    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None

    return div

def filter_cf(x, axis=1):
    count0 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    return count0/len(x)


In [6]:
MOD03_filepath = 'MYD03.A*.hdf'
MOD06_filepath = 'MYD06_L2.A*.hdf'
MOD03_filename, MOD06_filename =[],[]
MOD03_filename2, MOD06_filename2 =[],[]

for MOD06_filelist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_filelist, MOD06_filepath):
        MOD06_filename = MOD06_filelist
        MOD06_filename2.append(MOD06_filelist)

for MOD03_filelist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_filelist, MOD03_filepath):
        MOD03_filename = MOD03_filelist
        MOD03_filename2.append(MOD03_filelist)

if MOD03_filename and MOD06_filename:
    print('Reading Level 2 GeoLocation & Cloud Data')
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_filename,MOD03_path+MOD03_filename)

print('The Number Of Files In The MODO3 List: ')
print(len(MOD03_filename2))
print(' ')
print('The Number Of Files In The MODO6_L2 List: ')
print(len(MOD06_filename2))
print(' ')

Reading Level 2 GeoLocation & Cloud Data
 
 
The Number Of Files In The MODO3 List: 
3
 
The Number Of Files In The MODO6_L2 List: 
3
 


In [7]:
myd06_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/' #Passed While Code Execution.
#cm = np.zeros((2030,1354), dtype=np.float32)
cmfilelist = []

for MOD06_file in MOD06_filename2:
    MOD06_file2 = myd06_name + MOD06_file
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()

    #----#
    cm = da.from_array(CM, chunks =(2030,1354))
    #cm = da.from_array(CM, chunks =(500,500))
    cmfilelist.append(cm)
    cm = da.stack(cmfilelist, axis=0)


print('The Cloud Mask Array Shape Is: ',cm.shape)

myd03_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/' #Passed While Code Execution.


#lat = np.zeros((2030,1354), dtype=np.float32)
#lon = np.zeros((2030,1354), dtype=np.float32)
latfilelist = []
latfilelist2 = []
lonfilelist = []
lonfilelist2 = []

for MOD03_file in MOD03_filename2:
    MOD03_file2 = myd03_name + MOD03_file
    myd03 = Dataset(MOD03_file2, "r")

    #----#
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = da.from_array(latitude, chunks =(2030,1354))
    #lat = da.from_array(latitude, chunks =(500,500))
    latfilelist.append(lat)
    lat = da.stack(latfilelist, axis=0)
    '''
    latint = lat.astype(np.int8)
    latfilelist2.append(latint)
    latint = da.stack(latfilelist2, axis=0)
    '''


    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    lon = da.from_array(longitude, chunks =(2030,1354))
    #lon = da.from_array(longitude, chunks =(500,500))
    lonfilelist.append(lon)
    lon = da.stack(lonfilelist, axis=0)
    '''
    lonint = lon.astype(np.int8)
    lonfilelist2.append(lonint)
    lonint = da.stack(lonfilelist2, axis=0)
    '''


print('The Latitude Array Shape Is: ',lat.shape)
#print('The Latitude-Integer Array Shape Is: ',latint.shape)
print('The Longitude Array Shape Is: ',lon.shape)
#print('The Longitude-Integer Array Shape Is: ',lonint.shape)


The Cloud Mask Array Shape Is:  (3, 2030, 1354)
The Latitude Array Shape Is:  (3, 2030, 1354)
The Longitude Array Shape Is:  (3, 2030, 1354)


In [8]:
# Setting Integer Values Of Latitude & Longitude For GroupBy Operation.
latint = lat.astype(np.int8)
lonint = lon.astype(np.int8)

In [9]:
latint.size

8245860

In [10]:
dsa = xr.Dataset()
dsa.coords['Latitude'] = (('x','y','z'), lat)
dsa.coords['Longitude'] = (('x','y','z'), lon)
dsa.coords['LatInt'] = (('x','y','z'), latint)
dsa.coords['LonInt'] = (('x','y','z'), lonint)
dsa['CloudMask'] = (('x','y','z'), cm)

In [11]:
dsa

<xarray.Dataset>
Dimensions:    (x: 3, y: 2030, z: 1354)
Coordinates:
    Latitude   (x, y, z) float32 dask.array<shape=(3, 2030, 1354), chunksize=(1, 2030, 1354)>
    Longitude  (x, y, z) float32 dask.array<shape=(3, 2030, 1354), chunksize=(1, 2030, 1354)>
    LatInt     (x, y, z) int8 dask.array<shape=(3, 2030, 1354), chunksize=(1, 2030, 1354)>
    LonInt     (x, y, z) int8 dask.array<shape=(3, 2030, 1354), chunksize=(1, 2030, 1354)>
Dimensions without coordinates: x, y, z
Data variables:
    CloudMask  (x, y, z) int8 dask.array<shape=(3, 2030, 1354), chunksize=(1, 2030, 1354)>

In [13]:
list(dsa.groupby('LonInt'))

[(11, <xarray.Dataset>
  Dimensions:        (stacked_x_y_z: 3500)
  Coordinates:
      Latitude       (stacked_x_y_z) float32 dask.array<shape=(3500,), chunksize=(3500,)>
      Longitude      (stacked_x_y_z) float32 dask.array<shape=(3500,), chunksize=(3500,)>
      LatInt         (stacked_x_y_z) int8 dask.array<shape=(3500,), chunksize=(3500,)>
      LonInt         (stacked_x_y_z) int8 dask.array<shape=(3500,), chunksize=(3500,)>
    * stacked_x_y_z  (stacked_x_y_z) MultiIndex
    - x              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    - y              (stacked_x_y_z) int64 1587 1588 1589 1590 ... 1614 1615 1616
    - z              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  Data variables:
      CloudMask      (stacked_x_y_z) int8 dask.array<shape=(3500,), chunksize=(3500,)>),
 (12, <xarray.Dataset>
  Dimensions:        (stacked_x_y_z: 19843)
  Coordinates:
      Latitude       (stacked_x_y_z) float32 dask.array<shape=(19843,), chunksiz

In [14]:
# Performing GroupBy Operation To Obtain The Cloud Fractions.
start_time = time.time()
finallist = [] #Final List Of Latitude-Longitude-CM
dsag = list(dsa.groupby('LonInt'))
for listOfOneLong in dsag:
    oneGrid = listOfOneLong[1].groupby('LatInt').reduce(filter_cf)
    for index in range(0, oneGrid.CloudMask.size-1):
        finalvalues = [oneGrid.LatInt.data[index], listOfOneLong[0], oneGrid.CloudMask.data[index]]
        finallist.append(finalvalues)

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Total Time Taken This Loop:  75.21963000297546
00:01:15.22


In [16]:
finallist

[[16, 11, 0.03940886699507389],
 [17, 11, 0.0016835016835016834],
 [18, 11, 0.05241935483870968],
 [19, 11, 0.3212121212121212],
 [16, 12, 0.010260457774269928],
 [17, 12, 0.0],
 [18, 12, 0.16464285714285715],
 [19, 12, 0.5864745011086474],
 [20, 12, 0.5622775800711743],
 [21, 12, 1.0],
 [22, 12, 1.0],
 [23, 12, 0.99836867862969],
 [24, 12, 0.684964200477327],
 [25, 12, 0.8160676532769556],
 [15, 13, 0.0],
 [16, 13, 0.0],
 [17, 13, 0.001452643811737362],
 [18, 13, 0.17485679831172746],
 [19, 13, 0.3184653774173425],
 [20, 13, 0.18067632850241547],
 [21, 13, 0.8116666666666666],
 [22, 13, 1.0],
 [23, 13, 0.995709688952449],
 [24, 13, 0.6189948263118995],
 [25, 13, 0.32823259372609026],
 [26, 13, 0.7415287628053585],
 [27, 13, 0.9995553579368608],
 [28, 13, 0.34273318872017355],
 [29, 13, 0.16308724832214766],
 [30, 13, 0.6675392670157068],
 [31, 13, 0.9855072463768116],
 [32, 13, 0.9770114942528736],
 [33, 13, 0.5188284518828452],
 [15, 14, 0.0],
 [16, 14, 0.000471253534401508],
 [17, 1

In [15]:
cf_array = np.zeros((180,360))

# Creating Final List Of Latitude-Longitude-CM.
cf_array[:]=np.nan
print('The Length Of The Final List Of Latitude-Longitude-CM: ')
print(len(finallist))
print(' ')
print('-------------------------------------')
for i in range(len(finallist)):
    cf_array[(90-finallist[i][0]),(finallist[i][1]+180)] = finallist[i][2]

print(cf_array)

The Length Of The Final List Of Latitude-Longitude-CM: 
1776
 
-------------------------------------
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [17]:
# Read Total Cloud Fraction And GeoLocation Varibales To Create HDF5 Output File.
total_cloud_fraction = cf_array
out_name = 'output_final.hdf5'
def save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd):
    f=h5py.File(out_name,'w')
    PCentry=f.create_dataset('CF',data=total_cloud_fraction)
    PCentry.dims[0].label='lat_bnd'
    PCentry.dims[1].label='lon_bnd'

    PC=f.create_dataset('lat_bnd',data=lat_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Latitude_boundaries'

    PC=f.create_dataset('lon_bnd',data=lon_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Longitude_boundaries'
    f.close()
    print(out_name+' Saved!!')

save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd)


output_final.hdf5 Saved!!
